In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import time
import datetime
import os
import gc
import warnings
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split

In [2]:
op_train = pd.read_csv("../data/operation_train_new.csv")
trans_train = pd.read_csv("../data/transaction_train_new.csv")

In [3]:
op_test = pd.read_csv("../data/operation_round1_new.csv")
trans_test = pd.read_csv("../data/transaction_round1_new.csv")
y = pd.read_csv("../data/tag_train_new.csv")

D:\python3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
sub = pd.read_csv("../data/sub.csv")
def get_feature(op,trans,label):
    for feature in op.columns[2:]:
        label = label.merge(op.groupby(['UID'])[feature].count().reset_index(),on='UID',how='left')
        label =label.merge(op.groupby(['UID'])[feature].nunique().reset_index(),on='UID',how='left')
    
    for feature in trans.columns[2:]:
        if trans_train[feature].dtype == 'object':
            label =label.merge(trans.groupby(['UID'])[feature].count().reset_index(),on='UID',how='left')
            label =label.merge(trans.groupby(['UID'])[feature].nunique().reset_index(),on='UID',how='left')
        else:
            label =label.merge(trans.groupby(['UID'])[feature].count().reset_index(),on='UID',how='left')
            label =label.merge(trans.groupby(['UID'])[feature].nunique().reset_index(),on='UID',how='left')
            label =label.merge(trans.groupby(['UID'])[feature].max().reset_index(),on='UID',how='left')
            label =label.merge(trans.groupby(['UID'])[feature].min().reset_index(),on='UID',how='left')
            label =label.merge(trans.groupby(['UID'])[feature].sum().reset_index(),on='UID',how='left')
            label =label.merge(trans.groupby(['UID'])[feature].mean().reset_index(),on='UID',how='left')
            label =label.merge(trans.groupby(['UID'])[feature].median().reset_index(),on='UID',how='left')
            label =label.merge(trans.groupby(['UID'])[feature].std().reset_index(),on='UID',how='left')
            label =label.merge(trans.groupby(['UID'])[feature].skew().reset_index(),on='UID',how='left')
            
    return label


In [21]:
train = get_feature(op_train,trans_train,y).fillna(-1)
test = get_feature(op_test,trans_test,sub).fillna(-1)

In [22]:
train = train.drop(["UID", "Tag"], axis=1).fillna(value=-1)
label = y["Tag"]
test_id = test["UID"]


In [23]:
test = test.drop(["UID", "Tag"], axis=1).fillna(value=-1)

In [24]:
lgb_model = lgb.LGBMClassifier(boosting_type='gbdt',
                               num_leaves=64, 
                               reg_alpha=0,
                               reg_lambda=0, 
                               max_depth=-1,
                               n_estimators=1000, 
                               objective='binary', 
                               subsample=0.9, 
                               colsample_bytree=0.8, 
                               subsample_freq=1, 
                               learning_rate=0.05,
                               random_state=1024,
                               n_jobs=6,
                               min_child_weight=4, 
                               min_child_samples=5,
                               min_split_gain=0,
                               silent=True)

In [25]:
oof_preds = np.zeros(train.shape[0])
sub_preds = np.zeros(test_id.shape[0])

best_score = []
skf = StratifiedKFold(n_splits=5, random_state=2018, shuffle=True)
for index, (train_index, test_index) in enumerate(skf.split(train, label)):
    lgb_model.fit(train.iloc[train_index], label.iloc[train_index], verbose=50,
                  eval_set=[(train.iloc[train_index], label.iloc[train_index]),
                            (train.iloc[test_index], label.iloc[test_index])], early_stopping_rounds=100)
    best_score.append(lgb_model.best_score_['valid_1']['binary_logloss'])
    print(best_score)
    oof_preds[test_index] = lgb_model.predict_proba(train.iloc[test_index], num_iteration=lgb_model.best_iteration_)[:,1]

    test_pred = lgb_model.predict_proba(test, num_iteration=lgb_model.best_iteration_)[:, 1]
    sub_preds += test_pred / 5

Training until validation scores don't improve for 100 rounds.
[50]	valid_0's binary_logloss: 0.120584	valid_1's binary_logloss: 0.139646
[100]	valid_0's binary_logloss: 0.0827471	valid_1's binary_logloss: 0.119957
[150]	valid_0's binary_logloss: 0.0628112	valid_1's binary_logloss: 0.115597
[200]	valid_0's binary_logloss: 0.049279	valid_1's binary_logloss: 0.114954
[250]	valid_0's binary_logloss: 0.0392405	valid_1's binary_logloss: 0.115538
[300]	valid_0's binary_logloss: 0.0315262	valid_1's binary_logloss: 0.11692
Early stopping, best iteration is:
[210]	valid_0's binary_logloss: 0.0469464	valid_1's binary_logloss: 0.114748
[0.11474838209897542]
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's binary_logloss: 0.120963	valid_1's binary_logloss: 0.13472
[100]	valid_0's binary_logloss: 0.0824262	valid_1's binary_logloss: 0.117429
[150]	valid_0's binary_logloss: 0.062294	valid_1's binary_logloss: 0.115531
[200]	valid_0's binary_logloss: 0.0483557	valid_1's bin

In [26]:
sub['Tag'] = sub_preds

In [27]:
sub.to_csv("../submission/lgb_baseline.csv", index=False)

In [28]:
sub.shape

(31198, 2)

In [29]:
sub.head(2)

,UID,Tag
0,30000,0.031538
1,30001,0.058121


In [30]:
# stacking
sub6173 = pd.read_csv("../submission/0.6173.csv")
lgb_bsl = pd.read_csv("../submission/lgb_baseline.csv")
UID = sub6173["UID"]
prob = 0.5*sub6173["Tag"] + 0.5*lgb_bsl["Tag"]
result = pd.DataFrame({"UID":UID, "Tag": prob})

In [32]:
result.to_csv("../submission/stacking.csv", index=False)